In [ ]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute

#cluster_name = "GIVE_A_CLUSTER_NAME"
cluster_name = "cluster_vagner_MLEngAzure_Course1"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###

try:
    cpu_cluster=ComputeTarget(workspace=ws,name=cluster_name)
    print('Cluster already exists!')
except:
    compute_config = AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2",max_nodes=4)
    cpu_cluster = ComputeTarget(ws,cluster_name,compute_config)
cpu_cluster.wait_for_completion(show_output=True)

In [ ]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
parameter_space={
    '--C'=choice(0.01,0.1,1.0,10.,100.),
    "--max_iter"=choice(100,500,1000)
}
ps = RandomParameterSampling(parameter_space)### YOUR CODE HERE ###

# Specify a Policy
# search more on BanditPolicy
policy = BanditPolicy(evaluation_interval=1, slack_factor=0.15, slack_amount=None, delay_evaluation=5)
### YOUR CODE HERE ###

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory='.',
                            script='train.py',
                      # what is those arguments
                            arguments=['--arg1', arg1_val, '--arg2', arg2_val],
                            compute_target=cpu_cluster,
                            environment=sklearn_env)### YOUR CODE HERE ###

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,
                    hyperparameter_sampling=ps,
                    policy=policy,
                    primary_metric_name='validation_acc',
                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                    max_total_runs=4,
                    max_concurrent_runs=4)### YOUR CODE HERE ###

In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
### YOUR CODE HERE ###
run_hyperdrive=exp.submit(config=hyperdrive_config)
run_hyperdrive.wait_for_completion(show_output=True)
RunDetails(run_hyperdrive).show()

In [ ]:
import joblib
# Get your best run and save the model from that run.
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['accuracy'])
#print('\n learning rate:',parameter_values[3])
#print('\n keep probability:',parameter_values[5])
#print('\n batch size:',parameter_values[7])
### YOUR CODE HERE ###

os.makedirs('outputs', exist_ok=True)

joblib.dump(value=best_run, filename='outputs/best_model.pkl')

saved_model=best_run.register_model(model_name='bestmodel_hyperdrive',model_path='./outputs/best_model.pkl)

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
path='https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'
data_bank=TabularDatasetFactory.from_delimited_files(path)
### YOUR CODE HERE ###

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(data_bank)### YOUR DATA OBJECT HERE ###

# split into train and test data

x_train,x_test,y_train,y_test=train_test_split(x,y),train_size=0.7,random_state=42)

data_train=pd.concat([x_train,y_train])
data_test=pd.concat([x_test,y_test])


In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=data_train,
    label_column_name='y',
    n_cross_validations=5)

In [2]:
# Submit your automl run
run=exp.submit(automl_config, show_output=True)
run.wait_for_completion(show_output=True)
### YOUR CODE HERE ###

In [ ]:
RunDetails(run).show()

In [ ]:
# Retrieve and save your best automl model.
best_model = run.get_best_by_run_primary_metric()
best_model
### YOUR CODE HERE ###
# saving the model
os.makedirs('outputs', exist_ok=True)

joblib.dump(value=best_run, filename='outputs/best_model_automl.pkl')

saved_model=best_model.register_model(model_name='bestmodel_automl',model_path='./outputs/best_model_automl.pkl)

In [ ]:
# test the best model
from sklearn.metrics import accuracy_score
y_pred=best_model.predict(x_test)
acc=accuracy_score(y_pred,y_test)

In [ ]:
# deleting the compute cluster

cpu_cluster.delete()